In [ ]:
#비디오 프레임마다 이미지 추출
import os
import cv2

def video2frame(video_name, invideofilename, save_path):
    vidcap = cv2.VideoCapture(invideofilename)
    count = 0
    while True:
      success,image = vidcap.read()
      if not success:
          break
      if(int(vidcap.get(1)) % 10 == 0):
        print ('Read a new frame: ', success)
        fname = "{}.jpg".format("{0:05d}".format(count))
        cv2.imwrite(save_path + video_name + '_' + fname, image) # save frame as JPEG file
        count += 1
    print("{} images are extracted in {}.". format(count, save_path))


video_list = os.listdir('./kickboard_video')

for video_name in video_list:
    name = video_name.split('.')[0]
    video2frame(name, './kickboard_video/' + video_name, './images/')

In [ ]:
from data_aug.data_aug import *
from data_aug.bbox_util import *
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import pickle as pkl
%matplotlib inline

import os
from PIL import Image

img_path = './kickboard_yolo/train/images/' # 이미지 경로
label_path = './kickboard_yolo/train/labels/' # 라벨 경로
img_li = os.listdir(img_path)
label_li = os.listdir(label_path)

def convert(size, box): 
    dw = 1./size[1] 
    dh = 1./size[0] 
    x = (box[0] + box[2])/2.0 
    y = (box[1] + box[3])/2.0 
    w = box[2] - box[0] 
    h = box[3] - box[1] 
    x = x*dw 
    w = w*dw 
    y = y*dh 
    h = h*dh 
    return (x,y,w,h)

for i in range(len(img_li)):
    if img_li[i] == ".DS_Store":
        continue
    file_name = img_li[i].split('.')[0]
    image = './kickboard_yolo/train/images/' + img_li[i]
    print('이미지 경로', image)
    img = cv2.imread(image)[:,:,::-1]
    label_name = label_path + image.split('/')[-1].split('.')[0] + '.txt'
    print('라벨 경로', label_name)
    size = []
    size.append(img.shape[0]) 
    size.append(img.shape[1]) 
    print('이미지 크기', str(size))

    bboxes = np.loadtxt(label_name, delimiter=' ', skiprows=0, dtype=float)
    print('읽은 라벨 값 yolo', bboxes)

    if bboxes.ndim==1:
            rebboxes=np.zeros((1,bboxes.shape[0]))
    else:
        rebboxes=np.zeros(bboxes.shape)


    if bboxes.ndim==1:
        rebboxes[0][0]=bboxes[1]
        rebboxes[0][1]=bboxes[2]
        rebboxes[0][2]=bboxes[3]
        rebboxes[0][3]=bboxes[4]
        rebboxes[0][4]=bboxes[0]
        boundingbox=np.zeros(rebboxes.shape)
    else:
        rebboxes=bboxes[:,1:]
        rebboxes=np.hstack([rebboxes,bboxes[:,0:1]])
        boundingbox=np.zeros(bboxes.shape)

    print('순서 바꿈', rebboxes)

    #([0]=좌측상단 x, [1]=좌측상단 y, [2]=우측하단 x,[3]=우측하단 y)
    for j in range(boundingbox.shape[0]):
        boundingbox[j,0], boundingbox[j,1] = size[1] * (rebboxes[j,0] - rebboxes[j,2] / 2), size[0] * (rebboxes[j,1] - rebboxes[j,3] / 2)
        boundingbox[j,2], boundingbox[j,3] = size[1] * (rebboxes[j,0] + rebboxes[j,2] / 2), size[0] * (rebboxes[j,1] + rebboxes[j,3] / 2)
        boundingbox[j,4]=rebboxes[j,4]

    print('바운딩 박스', boundingbox)
    
    seq = Sequence([HorizontalFlip()])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_1" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_1" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([Rotate(5)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_2" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_2" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([Rotate(-5)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_3" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_3" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 30, 30)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_4" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_4" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), HorizontalFlip()])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_5" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_5" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), Rotate(10)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_6" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_6" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), Rotate(-10)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/train/images/'+ file_name + "_7" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/train/labels/' + file_name + "_7" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))

print("finish")

In [ ]:
from data_aug.data_aug import *
from data_aug.bbox_util import *
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import pickle as pkl
%matplotlib inline

import os
from PIL import Image

img_path = './kickboard_yolo/valid/images/' # 이미지 경로
label_path = './kickboard_yolo/valid/labels/' # 라벨 경로
img_li = os.listdir(img_path)
label_li = os.listdir(label_path)

def convert(size, box): 
    dw = 1./size[1] 
    dh = 1./size[0] 
    x = (box[0] + box[2])/2.0 
    y = (box[1] + box[3])/2.0 
    w = box[2] - box[0] 
    h = box[3] - box[1] 
    x = x*dw 
    w = w*dw 
    y = y*dh 
    h = h*dh 
    return (x,y,w,h)

for i in range(len(img_li)):
    if img_li[i] == ".DS_Store":
        continue
    file_name = img_li[i].split('.')[0]
    image = './kickboard_yolo/valid/images/' + img_li[i]
    print('이미지 경로', image)
    img = cv2.imread(image)[:,:,::-1]
    label_name = label_path + image.split('/')[-1].split('.')[0] + '.txt'
    print('라벨 경로', label_name)
    size = []
    size.append(img.shape[0]) 
    size.append(img.shape[1]) 
    print('이미지 크기', str(size))

    bboxes = np.loadtxt(label_name, delimiter=' ', skiprows=0, dtype=float)
    print('읽은 라벨 값 yolo', bboxes)

    if bboxes.ndim==1:
            rebboxes=np.zeros((1,bboxes.shape[0]))
    else:
        rebboxes=np.zeros(bboxes.shape)


    if bboxes.ndim==1:
        rebboxes[0][0]=bboxes[1]
        rebboxes[0][1]=bboxes[2]
        rebboxes[0][2]=bboxes[3]
        rebboxes[0][3]=bboxes[4]
        rebboxes[0][4]=bboxes[0]
        boundingbox=np.zeros(rebboxes.shape)
    else:
        rebboxes=bboxes[:,1:]
        rebboxes=np.hstack([rebboxes,bboxes[:,0:1]])
        boundingbox=np.zeros(bboxes.shape)

    print('순서 바꿈', rebboxes)

    #([0]=좌측상단 x, [1]=좌측상단 y, [2]=우측하단 x,[3]=우측하단 y)
    for j in range(boundingbox.shape[0]):
        boundingbox[j,0], boundingbox[j,1] = size[1] * (rebboxes[j,0] - rebboxes[j,2] / 2), size[0] * (rebboxes[j,1] - rebboxes[j,3] / 2)
        boundingbox[j,2], boundingbox[j,3] = size[1] * (rebboxes[j,0] + rebboxes[j,2] / 2), size[0] * (rebboxes[j,1] + rebboxes[j,3] / 2)
        boundingbox[j,4]=rebboxes[j,4]

    print('바운딩 박스', boundingbox)
    
    seq = Sequence([HorizontalFlip()])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_1" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_1" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([Rotate(5)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_2" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_2" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([Rotate(-5)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_3" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_3" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 30, 30)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_4" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_4" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), HorizontalFlip()])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_5" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_5" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), Rotate(10)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_6" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_6" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))
    
    seq = Sequence([RandomHSV(40, 40, 30), Rotate(-10)])
    # augmentation 과정
    img_, boundingbox_ = seq(img.copy(), boundingbox.copy())
    plotted_img = draw_rect(img_, boundingbox_)
    plt.imshow(plotted_img)
    yolobox=np.zeros((boundingbox_.shape))
    plt.show()

    # 라벨을 YOLO 형식으로 바꿔주는 과정
    for k in range(0,boundingbox_.shape[0]):
        x,y,w,h=convert(size,boundingbox_[k])
        yolobox[k][0]=boundingbox_[k][4]
        yolobox[k][1]=x
        yolobox[k][2]=y
        yolobox[k][3]=w
        yolobox[k][4]=h

    new_img_ = Image.fromarray(img_)
    # augmentation한 이미지와 라벨 저장
    print(new_img_.save('./kickboard_yolo/valid/images/'+ file_name + "_7" + '.jpg'))
    print(np.savetxt('./kickboard_yolo/valid/labels/' + file_name + "_7" + '.txt',yolobox, fmt='%d %f %f %f %f',delimiter=' '))

print("finish")